In [2]:
%matplotlib inline

import gc
import fpfs
import galsim

import numpy as np
import matplotlib.pyplot as plt
import numpy.lib.recfunctions as rfn

In [7]:
def analyze_FPFS(rng:np.random.RandomState, input_shear:np.ndarray, num_gals:int,\
        noi_stds,noi_psf=0.) -> tuple[np.ndarray,np.ndarray]:
    noi_stds=   np.array(noi_stds)
    if len(noi_stds.shape)==0:
        noi_stds=np.array([noi_stds])
    elif len(noi_stds.shape)>=2:
        raise ValueError('The input noi_stds should be float or 1d list')
    y       =   []
    y_err   =   []
    # initialize FPFS task with psf
    gal,psf =   fpfs.simutil.make_simple_sim(shear=[0.,0.],rng=rng,noise=0.)
    ngrid   =   psf.shape[0]
    rcut    =   16
    beg     =   ngrid//2-rcut
    end     =   beg+2*rcut
    psf     =   psf[beg:end,beg:end]
    gc.collect()
    num_tests=  noi_stds.size
    for i in range(num_tests):
        fpTask   =  fpfs.fpfsBase.fpfsTask(psf,noiFit=noi_stds[i]**2.,beta=0.75)
        results=[]
        for _ in range(num_gals):
            gal  =  fpfs.simutil.make_simple_sim(shear=input_shear,\
                    rng=rng,noise=noi_stds[i],psf_noise=noi_psf)[0]
            ngrid=  gal.shape[0]
            beg  =  ngrid//2-rcut
            end  =  beg+2*rcut
            gal  =  gal[beg:end,beg:end]
            modes=  fpTask.measure(gal)
            results.append(modes)
            del gal,modes
            gc.collect()
        mms =   rfn.stack_arrays(results,usemask=False)
        ells=   fpfs.fpfsBase.fpfsM2E(mms,const=2000,noirev=False)
        del mms,results
        resp=np.average(ells['fpfs_RE'])
        shear=np.average(ells['fpfs_e1'])/resp
        shear_err=np.std(ells['fpfs_e1'])/np.abs(resp)
        y.append(shear)
        y_err.append(shear_err)
    return np.array(y), np.array(y_err)

In [8]:
rng = np.random.RandomState(1024)
noise_arr = np.logspace(-4, -3, 5, base=10.0)
num_tests = len(noise_arr)
num_gals = 100
shear_true = [0.03, 0.00]
anz=analyze_FPFS(rng,shear_true,num_gals,noise_arr)

In [10]:
plt.close()
plt.eanz

(array([0.02958141, 0.03045444, 0.02989652, 0.03058665, 0.03278453]),
 array([0.00286062, 0.00559133, 0.01003668, 0.01835523, 0.02857018]))